# *k*-means clustering

*Supervised* machine learning (ML) is when we provide labeled training data for the algorithm. *k*-means clustering is an *unsupervised* ML algorithm: we provide the algorithm with a set of $n$ data points and tell the algorithm that we want that data partitioned into $k$ clusters that provide the best separation between clusters. 

The *k*-means algorithm randomly assigns $k$ *centroids* (or geometric centers of subsets of the dataset as measured through [Euclidean distance](http://mathworld.wolfram.com/Distance.html)) to the data and then iteratively moves those centroids around the feature space until it converges on an arrangement of the centroids the minimizes the variance within the clusters.

![SegmentLocal|20%](Images/K-means_convergence.gif "segment")

(Image courtesy of [Wikimedia Commons](https://en.wikipedia.org/wiki/File:K-means_convergence.gif) and is distributed under the terms of the [GNU Free Documentation License](https://en.wikipedia.org/wiki/GNU_Free_Documentation_License).)

> **Learning objective:** By the end of this section, you should have a basic understanding of the kinds of results the *k*-means algorithm can produce and have some idea about how you can interpret those results.

## Load and prepare data

To illustrate *k*-means clustering in action, we'll use the [U.S. Department of Agriculture National Nutrient Database for Standard Reference](https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/nutrient-data-laboratory/docs/usda-national-nutrient-database-for-standard-reference/) dataset. (Note that the path name is case sensitive.)

In [ ]:
import pandas as pd
df = pd.read_csv('Data/USDA-nndb-combined.csv', encoding='latin_1')

In [ ]:
df.head()

Because the *k*-means algorithm works with the Euclidean distance between data points, we need to separate out our descriptive from our numeric features (while saving the descriptive features for later use).

In [ ]:
desc_df = df.iloc[:, [0, 1, 2]+[i for i in range(50,54)]]
desc_df.set_index('NDB_No', inplace=True)
desc_df.head()

In [ ]:
nutr_df = df.iloc[:, :-5]
nutr_df = nutr_df.drop(['FoodGroup', 'Shrt_Desc'], axis=1)
nutr_df.set_index('NDB_No', inplace=True)
nutr_df.head()

The correlations that we have seen before in this dataset are still an issue here.

|   | column            | row               | corr |
|--:|------------------:|------------------:|-----:|
| 0 | Folate\_Tot\_(µg) | Folate\_DFE\_(µg) | 0.98 |
| 1 | Folic\_Acid\_(µg) | Folate\_DFE\_(µg) | 0.95 |
| 2 | Folate\_DFE\_(µg) | Folate\_Tot\_(µg) | 0.98 |
| 3 | Vit\_A\_RAE       | Retinol\_(µg)     | 0.99 |
| 4 | Retinol\_(µg)     | Vit\_A\_RAE       | 0.99 |
| 5 | Vit\_D\_µg        | Vit\_D\_IU        | 1    |
| 6 | Vit\_D\_IU        | Vit\_D\_µg        | 1    |

Before dropping anything from the `df` `DataFrame`, however, let's take a quick look at these correlations visually. Let's start with `Folate_Tot_(µg)` and `Folate_DFE_(µg)`.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.scatter(df['Folate_Tot_(Âµg)'], df['Folate_DFE_(Âµg)'])

This is a dramatic correlation!

> **Troubleshooting note:** Despite specifying Latin_1 encoding when we read in the dataset from the CSV file, character corruption can still occur. If the plt.scatter call above fails, try this code instead: `plt.scatter(df['Folate_Tot_(Âµg)'], df['Folate_DFE_(Âµg)'])`

> **Exercise**
>
> In the code cell below, run a scatterplot of another correlative pair of features from this dataset. (**Hint:** You only need to run the `plt.scatter` function with new parameters.)

Let's drop `Folate_DFE_(µg)`, `Vit_A_RAE`, and `Vit_D_IU` from `df` in order to eliminate these problematic correlations. The *k*-means algorithm also doesn't work with `NaN` values, so we will also have to drop those.

In [ ]:
nutr_df.drop(['Folate_DFE_(Âµg)', 'Vit_A_RAE', 'Vit_D_IU'], 
        inplace=True, axis=1)
nutr_df = nutr_df.dropna()
nutr_df.head()

> **Troubleshooting note:** Again, if the read_csv() function did not read in the µ (mu) symbol correctly and the drop() method call above fails, try this code instead: `nutr_df.drop(['Folate_DFE_(Âµg)', 'Vit_A_RAE', 'Vit_D_IU'], inplace=True, axis=1)`

Because the *k*-means algorithm will be calculating the Euclidean distances between data points and centroids, we need to ensure that all of the numeric features in our dataset use compatible units; we don't want to try and calculate distances between units of mass like grams and units of energy like kilocalories. We will use scikit-learn's [`StandardScaler()` function](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to center the data around each feature's mean and transform each feature to have a standard deviation of 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(nutr_df)

## Fitting the *k*-means clustering model

We are now ready to import *k*-means clustering algorithm.

In [ ]:
from sklearn.cluster import KMeans

While we can use any number of clusters that we wish, we will initially use three clusters as a convenient number for an initial exploration of k-means clustering.

In [ ]:
kmeansmodel = KMeans(n_clusters=3, random_state=13)
kmeansmodel.fit(X)

> **Question**
>
> What role does our declaration of the `random_state` above play in the `KMeans()` function?

Let's take a look at the labels for our clusters.

> **Question**
>
> Given what you know about Python zero-indexing and the number of clusters we selected, what labels do you expect to see?

In [ ]:
kmeansmodel.labels_

As expected, our labels for the different clusters are `0`, `1`, and `2`.

Let's examine the distribution of data points among the clusters.

In [ ]:
pd.value_counts(kmeansmodel.labels_)

You might also check various cluster stats for comparison of the clusters.

We will want to access these labels later on, so let's add them to our `DataFrame`.

In [ ]:
nutr_df['Cluster'] = kmeansmodel.labels_
nutr_df.head()

> **Takeaway:** Because data naturally clusters, we should not expect the clusters to be the same size.

## Interpreting the *k*-means clustering model

What sorts of foods populate the different clusters? We'll examine that question in greater depth later on, but we can get some sense of our clusters just by looking at the mean values for each of them.

In [ ]:
nutr_df.groupby('Cluster').mean()

We can quickly see that cluster 2 encompasses fatty foods (high `Lipid_Tot_(g)` values). Clusters 0 and 1 have similar mean protein and lipid amounts; an easy-to-see differentiation between those clusters is their relative carbohydrate and sugar levels, with those in cluster 1 being significantly higher than those in cluster 0.

We can look at these high-level differences in a little more detail using the `describe()` method, though, honestly, given the size of the `DataFrame`, this is a little cumbersome.

In [ ]:
nutr_df.groupby('Cluster').describe()

We can also predict the label for new or hypothetical observations. Let's provide the nutritional details for Gjetost cheese and see which cluster our model places it in. (Gjetost cheese is one of our discarded entries from the original `DataFrame` with `NaN` values imputed below.)

In [ ]:
newcase = [[1021,
            13.44,
            466.0,
            9.65,
            29.51,
            4.75,
            42.65,
            0.0,
            1.50,
            400.0,
            0.52,
            70.0,
            444.0,
            1409.0,
            600.0,
            1.14,
            0.08,
            0.04,
            14.5,
            0.0,
            0.315,
            1.382,
            0.813,
            3.351,
            0.271,
            5.0,
            0.0,
            5.0,
            15.4,
            2.42,
            1113.0,
            334.0,
            0,
            20,
            0,
            0.5,
            22,
            2.4,
            19.16,
            7.879,
            0.938,
            94.0,
            28.35]]

In [ ]:
kmeansmodel.predict(newcase)

The output indicates that Gjetost cheese falls under cluster 1.

> **Exercise**
>
> Now go back up and alter the values for Gjetost cheese  in the `newcase` array above to see what it takes to get that array classified into cluster 0 or cluster 2.

We can also see the actual coordinate values for cluster centroids.

In [ ]:
kmeansmodel.cluster_centers_

Remember that there is a method to this wall of numbers. Each coordinate is a point in 43-dimensional space, so each centroid is represented by an array of 43 values.

Let's now add back in our descriptive features so that we can see which food groups are in our clusters.

In [ ]:
merged_df = nutr_df.join(desc_df)
merged_df.head()

Now we can look at the value counts for the different food groups in our clusters.

In [ ]:
merged_df.loc[nutr_df['Cluster'] == 0]['FoodGroup'].value_counts()

> **Question**
>
> Look back over the Python in the code cell above. Does the syntax make sense? If not, review the documentation for [`pandas.DataFrame.loc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) and [`pandas.Series.value_counts`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).

> **Exercise**
>
> Now find the `FoodGroup` value counts for clusters 1 and 2.

> **Exercise solutions**

In [ ]:
merged_df.loc[nutr_df['Cluster'] == 1]['FoodGroup'].value_counts()

In [ ]:
merged_df.loc[nutr_df['Cluster'] == 2]['FoodGroup'].value_counts()

Some of the entries in our clusters makes sense and others look kind of like the contents of a grab bag. Part of the reason for this is that the *k*-means algorithm has to draw rather arbitrary boundaries between clusters; there will be a lot of entries in all clusters that are right on the edge and could reasonably belong to two (or more) clusters. To reduce some of this noise, we can sort these by distance from centroid of the respective clusters and look at the entries closest to the centroids.

In [ ]:
import numpy as np

distances = kmeansmodel.fit_transform(X)
merged_df['Distance'] = np.min(distances, axis=1)
merged_df.head()

> **Questions**
>
> 1. Why is it only necessary to find the minimum distance to the three centroids for a given data point (`np.min`)?
> 2. What role does the `axis=1` parameter play in the `np.min()` function?

We can now sort `FoodGroup` value counts by distance to the cluster's centroid, which can make the lists of principal food types in each cluster more intuitive.

In [ ]:
merged_df.loc[nutr_df['Cluster'] == 0].sort_values(by='Distance')['FoodGroup'][:500].value_counts()

> **Question**
>
> Look back over the Python in the code cell above. Does the syntax make sense? If not, review the documentation for [`pandas.DataFrame.sort_values`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) and [slicing ranges in pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#slicing-ranges).

> **Exercise**
>
> Now find the sorted `FoodGroup` value counts for clusters 1 and 2. (**Note:** Due to the differing sizes of the clusters, try a variety of ranges for your slices.)

> **Possible exercise solutions**

In [ ]:
merged_df.loc[nutr_df['Cluster'] == 1].sort_values(by='Distance')['FoodGroup'][:30].value_counts()

In [ ]:
merged_df.loc[nutr_df['Cluster'] == 2].sort_values(by='Distance')['FoodGroup'][:200].value_counts()

> **Takeaway:** Even without subject-matter expertise, we can gain insights of varying depth about the clusters.

## Visualizing the *k*-means clustering model

We can create visualizations of our clusters, but visualization is crude at best when it has been projected down through so many dimensions. That said, even trying to project from 43 dimensions down to 3 dimensions, noticeable patterns persist. To perform this visualization, we will first need to import the Axes3D module.

In [ ]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure(figsize=(6,6))
ax = Axes3D(fig)
C = kmeansmodel.cluster_centers_
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=kmeansmodel.labels_, alpha=0.3)

plt.show()

> **Takeaway:** Two observations:
> 1. Notice that there is no hard boundary visible between the purple and yellow clusters. The boundary is there, but, because it's a higher-dimensional hyperplane, it gets smeared across three dimensional space.
> 2. Note also that points from the light blue cluster are scattered across the plot. This is also an artifact of the clusters projection down from 43 dimensions to 3.

## Determining the number of clusters to use

You might have noticed that we manually determined the number $k$, the number of clusters, earlier in this section. We assigned `n_clusters_ = 3` in the model. But is 3 the right number of clusters for this dataset?

In practice, determining the correct number of clusters can require a lot on domain knowledge and data interpretation. However, we can use the elbow method as a rough heuristic for determining the number of clusters to use, even absent any domain knowledge.

In the elbow method, we plot the within-cluster sum of squares (WCSS, a measure of the variability of the observations within each cluster) against different values of $k$. We are looking for the value of $k$ near where WCSS begins to flatten out and form an elbow.

In [ ]:
WCSS = []
for num in range(1, 12):
    kmeans = KMeans(n_clusters = num,  random_state = 50)
    kmeans.fit(X)
    WCSS.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(13, 8))
    
plt.plot(range(1, 12), WCSS,'ro--',linewidth=2, markersize=12)
# plt.title('ELBOW METHOD')
plt.rcParams.update({'font.size': 14})
plt.xlabel('Number of clusters (k)')
plt.ylabel('WCSS')
plt.show()

> **Takeaway:** Ideally, we would want to see a more pronounced elbow. One could make the case for the elbow being at $k=5$ — or at $k=8$ or $k=10$. And some datasets do produce a distinct elbow. However, real-world data is inherently messy, and there is no perfect way to evaluate *k*-means models apart from domain expertise and rigorous analysis.